In [78]:
from bs4 import BeautifulSoup #parser - can be made to crawl through while True + code as shown below
import requests #easilly send http requests to retrieve 
import time

In [79]:
url = "https://boston.craigslist.org/search/ofc" #page we want to scrape and start crawling from

In [80]:
def get_soup(url):
    response = requests.get(url)
    data = response.text
    return BeautifulSoup(data, 'html.parser')

In [81]:
while True:
    
    soup = get_soup(url)
    jobs = soup.find_all('div', {'class':'result-info'})

    for job in jobs: #scrapes job offers in from page(url)

        title = job.find('a',{'class':'result-title'}).text
        location_tag = job.find('span', {'class':'result-hood'})
        location = location_tag.text[2:-1].strip() if location_tag else job.find('span', {'class':'nearby'}).text
        date = job.find('time', {'class':'result-date'}).text
        link = job.find('a',{'class':'result-title'}).get('href')

        description_response = requests.get(link)
        description_soup = BeautifulSoup(description_response.text, 'html.parser')
        description = description_soup.find('section', {'id':'postingbody'}).text

        print(f'Job title : {title}\nLocation : {location}\nDate : {date}\nLink : {link}\nDescription : {description}\n\n\n---\n\n\n')
    

    next_page_tag = get_soup(url).find('a', {'class':'button next'}) # next page div

    if next_page_tag.get('href'): # if there is a link, update url to crawl next page
        url = 'https://boston.craigslist.org' + next_page_tag.get('href')
    else:
        break

time to get soup is 0.7367708683013916 seconds
time to find_all is 0.008003711700439453 seconds
Job title : PT Jack of all trades
Location : Hudson metro west
Date : Dec 26
Link : https://boston.craigslist.org/bmw/ofc/d/hudson-pt-jack-of-all-trades/7425013381.html
Description : 

QR Code Link to This Post


A small tax/business office in Hudson area that is in growing pains mode and is looking for PT Jack of All Trades.  We do all types of returns and deal with clients who have IRS tax problems as well. Prefer a candidate that can work multiple projects, maintaining individual client needs (hand holding) and is a good multi-tasker. We tend to attract people with issues, so cleaning up messes and saving clients from themselves is not uncommon. 10-20 hours (off season)/15+ hours (Feb-April) a week is needed with definite possibility for more in the future and flexible hours are available. We are a scanning office that uses a client portal and Document Management system. COVID: Because of

***requests.get*** takes about .5 seconds - it is called **once per page** but more importantly **once per offer**

***find*** takes virtually no time

time to get job description takes .35 which is roughly equal to the request time (parse + find pretty quick) 

*We need to parallel process requesting the descriptions of each offer within a page*

After that, we might parallel process each page's scraping

**multithread request descriptions <=> multithread for job in jobs loop**

# Best way seems to first get all the urls of a page in a list and follow https://beckernick.github.io/faster-web-scraping-python/